## Escenarios de migración interna

In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# funciones desarrolladas
from functions.agrupar_dfs_censo import *
from functions.cargar_data import *
from functions.impresion import *

In [5]:
prodSim = sm.load('modelos/restringido_origen_mvo.pickle')
prodSim

In [6]:
# carga datos diádicos
dd_deptos = cargar_dd_deptos()
print(dd_deptos.shape)
print(dd_deptos.columns)
dd_deptos.head()

(342, 46)
Index(['cod', 'depto_origen', 'depto_destino', 'personas_mig',
       'personas_mig_18_25', 'personas_mig_26_35', 'personas_mig_100k',
       'pob_origen', 'pob_destino', 'dist', 'dist_km', 'pbi_origen',
       'pbi_porcen_ori', 'pbi_destino', 'pbi_porcen_des',
       'pbi_origen_millardos', 'pbi_destino_millardos', 'largo_limite',
       'dummy_limit', 'empresas_origen', 'empresas_destino',
       'menores_25_origen', 'menores_25_destino', 'pob_destino_k',
       'pob_origen_k', 'nom_depto_orig', 'nom_depto_des', 'edad_prom_ori',
       'porc_ocupados_ori', 'edad_prom_des', 'porc_ocupados_des',
       'tasa_desemp_mujeres_2011_ori', 'tasa_desemp_varones_2011_ori',
       'tasa_desemp_total_2011_ori', 'tasa_desemp_mujeres_2011_des',
       'tasa_desemp_varones_2011_des', 'tasa_desemp_total_2011_des',
       'log_pbi_destino', 'log_dist_km', 'log_pbi_origen_millardos',
       'log_pbi_destino_millardos', 'log_pob_destino_k', 'log_pob_origen_k',
       'log_edad_prom_des', 'log

,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35,personas_mig_100k,pob_origen,pob_destino,dist,...,tasa_desemp_total_2011_des,log_pbi_destino,log_dist_km,log_pbi_origen_millardos,log_pbi_destino_millardos,log_pob_destino_k,log_pob_origen_k,log_edad_prom_des,log_porc_ocupados_des,largo_limite_km
0,102,1,2,914,114,247,891,1318755,73377,610726,...,8.1,16.469807,6.415097,6.143859,2.654298,4.295611,7.184443,3.525387,4.151677,1.000000e-07
1,302,3,2,74,9,22,70,520173,73377,601915,...,8.1,16.469807,6.400257,4.457899,2.654298,4.295611,6.254161,3.525387,4.151677,1.000000e-07
2,402,4,2,15,0,6,14,84698,73377,406694,...,8.1,16.469807,6.008813,2.836092,2.654298,4.295611,4.439092,3.525387,4.151677,1.000000e-07
3,502,5,2,28,7,8,25,123203,73377,556292,...,8.1,16.469807,6.320768,3.779383,2.654298,4.295611,4.813833,3.525387,4.151677,1.000000e-07
4,602,6,2,32,0,10,29,57084,73377,446123,...,8.1,16.469807,6.100319,2.566794,2.654298,4.295611,4.044524,3.525387,4.151677,1.000000e-07


# 1. Escenario 1

Describir

In [7]:
pbi = recuperar_crecimiento_pbi()
pbi

,año,pbi_porc
0,2015,0.37
1,2016,1.69
2,2017,1.63
3,2018,0.48
4,2019,0.35
5,2020,-6.00
6,2021,2.00


In [8]:
crecimiento_acum = round(pbi.pbi_porc.sum(), 3)
print('Crecimiento acumulado del PBI entre 2015 y 2021: ', crecimiento_acum)

Crecimiento acumulado del PBI entre 2015 y 2021:  0.52


In [9]:
pbi.pbi_porc.values

array([ 0.37,  1.69,  1.63,  0.48,  0.35, -6.  ,  2.  ])

In [10]:
g = pbi.pbi_porc.values/100
g

array([ 0.0037,  0.0169,  0.0163,  0.0048,  0.0035, -0.06  ,  0.02  ])

In [11]:
def calc_pbi(series, pbi_porc):
    "Estima PBI en base a crecimiento anual reportado"
    _2015 = series + (series * pbi_porc[0])
    _2016 = _2015 + (_2015 * pbi_porc[1])
    _2017 = _2016 + (_2016 * pbi_porc[2])
    _2018 = _2017 + (_2017 * pbi_porc[3])
    _2019 = _2018 + (_2018 * pbi_porc[4])
    _2020 = _2019 + (_2019 * pbi_porc[5])
    _2021 = _2020 + (_2020 * pbi_porc[6])

    return round(_2021).astype(int)

In [12]:
pbi_dest = dd_deptos.drop_duplicates('depto_destino')
pbi_dest = pbi_dest.loc[:, ['depto_destino', 'pbi_destino_millardos']]
pbi_dest.head()

,depto_destino,pbi_destino_millardos
0,2,14.215
18,3,86.306
36,4,17.049
54,5,43.789
72,6,13.024


In [13]:
# proyecta PBI
pbi_dest['pbi_destino_millardos_2021'] = calc_pbi(pbi_dest.pbi_destino_millardos, g)
pbi_dest.tail()


,depto_destino,pbi_destino_millardos,pbi_destino_millardos_2021
252,16,26.931,27
270,17,22.994,23
288,18,18.094,18
306,19,11.721,12
324,1,465.848,467


In [14]:
pbi_dest.drop('pbi_destino_millardos', inplace=True, axis=1)
dd_deptos = dd_deptos.merge(pbi_dest, left_on='depto_destino', right_on='depto_destino')
dd_deptos['log_pbi_destino_millardos'] = np.log(dd_deptos.pbi_destino_millardos_2021)
dd_deptos.head()

,cod,depto_origen,depto_destino,personas_mig,personas_mig_18_25,personas_mig_26_35,personas_mig_100k,pob_origen,pob_destino,dist,...,log_pbi_destino,log_dist_km,log_pbi_origen_millardos,log_pbi_destino_millardos,log_pob_destino_k,log_pob_origen_k,log_edad_prom_des,log_porc_ocupados_des,largo_limite_km,pbi_destino_millardos_2021
0,102,1,2,914,114,247,891,1318755,73377,610726,...,16.469807,6.415097,6.143859,2.639057,4.295611,7.184443,3.525387,4.151677,1.000000e-07,14
1,302,3,2,74,9,22,70,520173,73377,601915,...,16.469807,6.400257,4.457899,2.639057,4.295611,6.254161,3.525387,4.151677,1.000000e-07,14
2,402,4,2,15,0,6,14,84698,73377,406694,...,16.469807,6.008813,2.836092,2.639057,4.295611,4.439092,3.525387,4.151677,1.000000e-07,14
3,502,5,2,28,7,8,25,123203,73377,556292,...,16.469807,6.320768,3.779383,2.639057,4.295611,4.813833,3.525387,4.151677,1.000000e-07,14
4,602,6,2,32,0,10,29,57084,73377,446123,...,16.469807,6.100319,2.566794,2.639057,4.295611,4.044524,3.525387,4.151677,1.000000e-07,14


In [15]:
pbi_dest.to_csv('tablas/pbi_2021.csv', index=False)

In [16]:
pred = prodSim.get_prediction(dd_deptos)

In [17]:
dd_deptos['prodsimest'] = pred.summary_frame()['mean'].round(0).astype(int)
dd_deptos['prodsimest'] 

0       584
1        84
2        96
3        57
4        52
       ... 
337    2314
338    1779
339    2009
340    1217
341    1693
Name: prodsimest, Length: 342, dtype: int32

In [18]:
# matriz de los valores estimados
pd.pivot_table(dd_deptos,
values='prodsimest',
index ='depto_origen',
columns='depto_destino',
fill_value=0,
aggfunc=sum,
margins=True,
margins_name='Total')

depto_destino,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,Total
depto_origen,,,,,,,,,,,,,,,,,,,,
1,0,584,32017,825,2676,1042,529,1924,1295,4962,1389,1121,967,966,1301,4221,1348,980,674,58821
2,1693,0,579,139,206,103,49,106,80,270,273,170,355,89,439,166,159,215,82,5173
3,13420,84,0,122,356,153,78,303,204,751,200,160,139,145,187,517,193,142,101,17255
4,1641,96,580,0,165,102,42,104,99,317,150,103,197,118,176,148,110,156,163,4467
5,2142,57,680,66,0,95,63,112,71,248,165,155,90,64,144,253,215,92,48,4760
6,1524,52,533,75,173,0,71,122,70,212,128,98,91,61,117,156,116,105,59,3763
7,627,20,219,25,93,57,0,46,24,80,61,48,33,21,50,78,61,38,19,1600
8,1932,37,725,53,141,84,39,0,75,231,89,68,62,55,81,188,78,66,44,4048
9,1562,33,588,60,107,57,25,90,0,357,71,53,56,76,67,122,61,55,56,3496


# 2. Escenario 2

In [ ]:
d